In [1]:
# # Byte Pair Encoder
# import re
# from collections import defaultdict
# training_data = open("scifi.txt", "r")
# text = training_data.read()
# charset = "".join(sorted(set(text)))

# tokens = re.split(r'([\(\)\[\].:,!\s])', text)
# tokens = list(filter(lambda x: x != "", tokens))

# tokens = [list(token) for token in tokens]
# k = 90

# token_map = defaultdict(int)

# # prepare initial tokenset of purely characters
# for ch in text:
#     token_map[ch] += 1


# for _ in range(k):
#     updated_lst = []
#     bpe_table = defaultdict(int)
#     # find the highest frequency byte
#     for token in tokens:
#         if len(token) == 1:
#             continue
#         for i in range(len(token) - 1):
#             bpe_table["".join(token[i:i+2])] += 1

#     # get maximum frequency byte
#     max_bp = max(bpe_table, key=bpe_table.get)

#     for token in tokens:
#         temp_token = []
#         if len(token) == 1:
#             updated_lst.append(token)
#             continue

#         cont_flag = 0

#         # go over each character in token
#         for i in range(len(token)):
#             # avoid repeating tokens        
#             if cont_flag:
#                 cont_flag = 0
#                 continue
#             if i != len(token) - 1:
#                 pair = token[i:i+2]
#                 # print(pair)
#                 if "".join(pair) == max_bp:
#                     temp_token.append(max_bp)
#                     token_map[max_bp] += 1
#                     token_map[pair[0]] -= 1
#                     token_map[pair[1]] -= 1
#                     cont_flag = 1
#                     continue

#             # else add character
#             temp_token.append(token[i])
#             token_map[token[i]] += 1
            
#         updated_lst.append(temp_token)

#     # set tokens to updated_lst of merged token
#     tokens = updated_lst

In [2]:
# token_set = sorted(list(set([token_key for token_key in token_map if token_map[token_key] > -1])))
# print(token_set)

# # save token_set to file
# with open("token_set_scifi.txt", "w") as f:
#     f.write("±".join(token_set))

# load token_set from file
with open("token_set_scifi.txt", "r") as f:
    token_set = f.read().split("±")

# print(token_set)

# from itertools import chain

# bpe_text_in = list(chain.from_iterable(updated_lst))

# training_data = open("family-guy.txt", "r")
# # training_data = training_data.read()
# # charset = "".join(sorted(set(training_data)))

# def encoder(text_in):
#     pass
            

encoder = lambda text_in: [token_set.index(s) for s in text_in]
decoder = lambda indices: [token_set[index] for index in indices]
# device = "mps"

# token_set
# print(token_map)
# token_set[-40:]
# encoder(bpe_text_in[])

# encoder("hello")
# print(bpe_text_in)

In [3]:
import torch
from random import randint
import numpy as np

# context window of 380 tokens
block_size = 1024
batch_size = 20
vocab_size = len(token_set)
dropout = 0.2
n_heads = 8
n_embed = 512
num_blocks = 6
n_steps = 1200

device = "mps"


# data = torch.tensor(encoder(bpe_text_in), dtype=torch.long)
# save tensor to file 
# torch.save(data, 'data_scifi.pt')
# # load tensor from file
# data = torch.load('data_scifi.pt')
data = torch.load("data_scifi.pt")
data = data.to(device)

n = int(0.5*len(data))
training_set = data[:n]
validation_set = data[n:]


def generate_batch():
        randlst = torch.randint(len(training_set) - block_size, (batch_size,))#.to(device="mps")
        batch = torch.stack([training_set[i:i+block_size] for i in randlst])#.to(device="mps")
        targets = torch.stack([training_set[i+1: i+block_size+1] for i in randlst])#.to(device="mps")
        return batch,targets

def generate_batch_validation():
        randlst = torch.randint(len(validation_set) - block_size, (batch_size,))#.to(device="mps")
        batch = torch.stack([validation_set[i:i+block_size] for i in randlst])#.to(device="mps")
        targets = torch.stack([validation_set[i+1: i+block_size+1] for i in randlst])#.to(device="mps")
        return batch,targets

# x,y = generate_batch()

In [4]:
from FlashMetal import FlashAttentionForward, FlashAttentionBackward, fetchPipeline

# forward_pipeline, back_pipeline = fetchPipeline()

class FlashAttentionAutograd(torch.autograd.Function):
    @staticmethod
    def forward(ctx, query, key, value):
        batch_size, num_heads, N_seq, n_embed = query.shape

        out = torch.zeros_like(value, requires_grad=True, device='mps', dtype=torch.float32).contiguous()
        row_max = torch.zeros((batch_size, num_heads, N_seq), device='mps', dtype=torch.float32).contiguous()
        row_sum = torch.zeros((batch_size, num_heads, N_seq), device='mps', dtype=torch.float32).contiguous()

        # out = FlashAttentionForward(query, key, value, out, row_max, row_sum)#, forward_pipeline)
        out, row_max, row_sum = FlashAttentionForward(query, key, value, out, row_max, row_sum )        
        ctx.save_for_backward(query, key, value, out, row_max, row_sum)
        return out

    @staticmethod
    def backward(ctx, grad_output):
        query, key, value, out, row_max, row_sum = ctx.saved_tensors
        out_dQ = torch.zeros_like(query, device='mps', dtype=torch.float32).contiguous()
        out_dK = torch.zeros_like(key, device='mps', dtype=torch.float32).contiguous()
        out_dV = torch.zeros_like(value, device='mps', dtype=torch.float32).contiguous()
        res_metal = FlashAttentionBackward(query, key, value, out, grad_output.contiguous(), out_dQ, out_dK, out_dV, row_sum, row_max)
        grad_query, grad_key, grad_value = res_metal
        return grad_query, grad_key, grad_value

        # s = query @ key.transpose(-1,-2)
        # s /= np.sqrt(16)

        # mask = torch.tril(torch.ones_like(s)).to("mps")
        # s_masked = torch.where(mask == 1, s, torch.tensor(float('-inf')).to("mps"))

        # P = F.softmax(s_masked, -1)

        # dV = torch.matmul(P.transpose(-1, -2), grad_output)

        # dP = torch.matmul(grad_output, value.transpose(-1, -2))
        # dS = P * (dP - torch.sum(dP * P, dim=-1, keepdim=True))

        # dQ = (torch.matmul(dS, key))
        # dK = torch.matmul(dS.transpose(-1, -2), query)
        # return dQ, dK, dV



In [5]:
from torch import nn
from torch.nn import functional as F

class MHAttention(nn.Module):
    def __init__(self):
        super().__init__()
        self.head_size = 16
        self.batch_qkv_matrices = nn.Linear(n_embed, self.head_size * n_heads * 3, bias=False)
        self.projection = nn.Linear(self.head_size * n_heads, n_embed)
        self.dropout = nn.Dropout(dropout)
        self.kv_cache = None
    def forward(self, x):
        B,T,C = x.shape
        q,k,v = self.batch_qkv_matrices(x).split(self.head_size * n_heads, dim=-1) # Now Q,K,V of dim B, T, head size * n_heads

        T_Q = T
        T_KV = T

        if not self.training:
            if self.kv_cache is not None: 
                # concat new kv onto old kv cache -> update cache
                k_prev,v_prev = self.kv_cache
                # reset cache if block size reached
                if self.kv_cache[0].shape[-2] < block_size:
                    k = torch.cat((k_prev, k), dim=1)
                    v = torch.cat((v_prev, v), dim=1)
                    # trim key-value cache to block size
                    # k = k[:, -block_size:, :]
                    # v = v[:, -block_size:, :]
                    # set T_Q to 1 as we have 1-d query
                T_Q = 1
            T_KV = k.shape[-2]            
            self.kv_cache = (k,v)


        q = q.view(B, T_Q, n_heads, self.head_size).transpose(1,2).contiguous() # Now of shape B, n_heads, T, head_size for BMM
        k = k.view(B, T_KV, n_heads, self.head_size).transpose(1,2).contiguous()
        v = v.view(B, T_KV, n_heads, self.head_size).transpose(1,2).contiguous()


        if self.training:
            out = FlashAttentionAutograd.apply(q,k,v)
            out = out.transpose(1,2) # B, n_heads, T, C --> B, T, n_heads, C   
            out = out.contiguous().view(B,T, n_heads * self.head_size)
            return self.dropout(self.projection(out))

        # for inference
        weight_mat = q @ k.transpose(-2, -1)
        weight_mat = weight_mat * (self.head_size ** -0.5) 
        weight_mat = F.softmax(weight_mat, dim=-1)
        # Multiply with values
        res = weight_mat @ v
        res = res.transpose(1,2) # B, n_heads, T, C --> B, T, n_heads, C   
        res = res.contiguous().view(B, T, n_heads*self.head_size)
        return self.projection(res)

 
     

# class Head(nn.Module):
#     def __init__(self, head_size):
#         super().__init__()
#         # print(head_size)
#         self.head_size = head_size
#         self.batch_qkv_matrices = nn.Linear(n_embed, head_size * n_heads * 3, bias=False) 
#         self.register_buffer("tril", torch.tril(torch.ones((block_size, block_size))))
#         self.dropout = nn.Dropout(dropout)
#         self.kv_cache = None
    
#     # def reset_cache(self):
#     #     self.kv_cache = None

    # def forward(self, x):
    #     # initially tensor of B,T,C
    #     q,k,v = self.batch_qkv_matrices(x).split(self.head_size * n_heads, dim=-1) # Now Q,K,V of dim B, T, head size * n_heads
        
    #     B,T,C = x.shape
    #     # # assert C % n_heads == 0
    #     T_Q = T
    #     T_KV = T
    #     # if model in inference (eval) mode then use cache
        # if not self.training:
        #     if self.kv_cache is not None: 
        #         # concat new kv onto old kv cache -> update cache
        #         k_prev,v_prev = self.kv_cache
        #         # reset cache if block size reached
        #         if self.kv_cache[0].shape[-2] < block_size:
        #             k = torch.cat((k_prev, k), dim=1)
        #             v = torch.cat((v_prev, v), dim=1)
        #             # trim key-value cache to block size
        #             # k = k[:, -block_size:, :]
        #             # v = v[:, -block_size:, :]

        #         # set T_Q to 1 as we have 1-d query
        #         T_Q = 1
            
        #     T_KV = k.shape[-2]            
        #     self.kv_cache = (k,v)

 
#         k = k.view(B, T_KV, n_heads, self.head_size).transpose(1,2)
#         # # print(f"K: {k.shape}")
#         q = q.view(B, T_Q, n_heads, self.head_size).transpose(1,2) # Now of shape B, n_heads, T, head_size for BMM
#         v = v.view(B, T_KV, n_heads, self.head_size).transpose(1,2)
   
#         weight_mat = q @ k.transpose(-2, -1)

#         weight_mat = weight_mat * (self.head_size ** -0.5) #

#         if self.training:
#             weight_mat = weight_mat.masked_fill(self.tril[:T, :T] == 0, float("-inf"))

#         weight_mat = F.softmax(weight_mat, dim=-1)
#         # # regularisation prevent overfitting
#         weight_mat = self.dropout(weight_mat)
#         # Multiply with values
#         res = weight_mat @ v
#         res = res.transpose(1,2) # B, n_heads, T, C --> B, T, n_heads, C   
#         res = res.contiguous().view(B, T, self.head_size*n_heads)
#         return res
    
    # COMMENTED OUT FOR TESTING


# class MHAttention(nn.Module):
#     def __init__(self, num_heads, head_size):
#         super().__init__()
#         self.att_heads = Head(head_size=head_size) #nn.ModuleList([Head(head_size) for _ in range(num_heads)])
#         self.projection = nn.Linear(head_size*n_heads, n_embed)
#         self.dropout = nn.Dropout(dropout)
#     def forward(self, x):
#         # print(x.shape)
#         res = self.att_heads(x)
#         # res = torch.cat([att_head(x) for att_head in self.att_heads], dim=-1)
#         res = self.dropout(self.projection(res))
#         return res 
    

# hd = Head(96).to("mps")
# res = hd(torch.randn([64, 1024, 96*16], device="mps"))

# mh = MHAttention().to("mps")
# x = torch.randn(1,1024,n_embed, device='mps')
# loss = torch.mean(out)
# # # print(out.shape)
# # loss = torch.mean(out)
# loss.backward()



In [6]:
class Feedforward(nn.Module):
    def __init__(self, n_embed) -> None:
        super().__init__()
        scale_factor = 4
        self.ff = nn.Sequential(
            nn.Linear(n_embed, n_embed * scale_factor),
            nn.ReLU(),
            nn.Linear(n_embed * scale_factor, n_embed),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.ff(x)


class Block(nn.Module):
    def __init__(self, n_embed, n_heads) -> None:
        super().__init__()
        self.ff = Feedforward(n_embed)
        # self.mhatt = MHAttention(n_heads, 16)
        self.mhatt = MHAttention().to(device)
        self.layer_norm1 = nn.LayerNorm(n_embed) 
        self.layer_norm2 = nn.LayerNorm(n_embed)
    def forward(self, x):
        x = x + self.mhatt(self.layer_norm1(x))
        x = x + self.ff(self.layer_norm2(x))
        return x 
    


In [7]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size, n_embed)
        self.pos_embedding_table = nn.Embedding(block_size, n_embed)
       
        # self.att = Head(n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_heads) for _ in range(num_blocks)])

        self.layernorm =  nn.LayerNorm(n_embed)
        
    def forward(self, data, target=None, time_inf=None):
        # print(data.shape)
        token_layer = self.embedding_table(data)
        # print(token_layer)
        # print(token_layer.shape)
        B,T = data.shape


        if time_inf != None and T == 1:
            # print(torch.ones(1, dtype=torch.long) * time_inf)
            # pos_embed arguement must be tensor, must be single value of time_inf 
            pos_embed = self.pos_embedding_table(torch.ones(1, dtype=torch.long).to(device) * time_inf)
        else:
            # print(torch.arange(T))
            pos_embed = self.pos_embedding_table(torch.arange(T).to(device))
        
        total = token_layer + pos_embed
        # print(f"INPUT {total.shape}")

        total = self.layernorm(self.blocks(total))
        logits = self.embedding_table(total)

        if target != None:
            B,T,C = logits.shape
            logits = logits.view(B*T, C) 
            target = target.view(B*T)
            loss = F.cross_entropy(logits, target)
            return logits, loss
        return logits, None
        

    def predict(self, data, temperature=1, num_tokens=350):
        prev_token = data
        for i in range(num_tokens):
            t_curr = (i+data.shape[-1]-1) % block_size
            logits,lossNone = self(prev_token, time_inf=t_curr)
            logits = logits[:,-1,:]
            logits = logits / temperature
            prob_dist = F.softmax(logits, dim=-1)
            prev_token = torch.multinomial(prob_dist, num_samples=1)
            # print(prev_token)
            print(decoder(prev_token[0].tolist())[0],end="")



In [67]:
# training loop
from tqdm import tqdm
torch.autograd.set_detect_anomaly(True)
gpt = GPT().to("mps")
gpt.load_state_dict(torch.load("scifi_model_checkpoint.pth"))
gpt.train()
optimiser = torch.optim.AdamW(gpt.parameters(), lr=6e-4, betas=(0.9, 0.95), eps=1e-8, weight_decay=0.1)
for steps in tqdm(range(n_steps)):
    batch,target = generate_batch()
    logits, loss = gpt(batch,target)
    optimiser.zero_grad()
    loss.backward() 
    torch.nn.utils.clip_grad_norm_(gpt.parameters(), 1.0)
    optimiser.step()

    if steps % 200 == 0:
        # print(gpt.blocks[0].mhatt.batch_qkv_matrices.weight)
    #     # validation_batch, validation_target = generate_batch_validation()
    #     # validation_logits, validation_loss = gpt(validation_batch,validation_target)
        print(f"Training Loss: {loss.item()}")#. Validation Loss: {validation_loss.item()}")

  0%|          | 1/1200 [00:05<1:49:46,  5.49s/it]

Training Loss: 1.8558117151260376


 17%|█▋        | 201/1200 [13:19<1:05:29,  3.93s/it]

Training Loss: 1.7938086986541748


 33%|███▎      | 401/1200 [26:26<52:38,  3.95s/it]  

...

 67%|██████▋   | 801/1200 [52:59<26:24,  3.97s/it]

Training Loss: 1.788804292678833


 83%|████████▎ | 1001/1200 [1:06:10<13:06,  3.95s/it]

Training Loss: 1.7808431386947632


100%|██████████| 1200/1200 [1:19:14<00:00,  3.96s/it]


In [73]:
# torch.save(gpt.state_dict(), "scifi_model_checkpoint.pth")

def get_prompt(input_str):
    lst = []
    # we are iterating through each character
    # for each character, we check if it + curr_token is in token_set
    # if it is, then we continue 
    curr_tok = input_str[0]
    for i in range(1, len(input_str)):
        if (curr_tok + input_str[i]) not in token_set:
            lst.append(curr_tok)
            curr_tok = input_str[i]
            continue
        else:
            curr_tok += input_str[i]

    lst.append(curr_tok)
    return lst


gpt_eval = GPT().to(device)       
gpt_eval.load_state_dict(torch.load("scifi_model_checkpoint.pth"))

prompt = "\"Hello\", said John "
encoded_prompt = encoder(get_prompt(prompt))

test = torch.tensor(encoded_prompt ,dtype=torch.long, device=device)
# # # print(test)
# # # # # # # print(test.shape)
K = 1
test = test.unsqueeze(0).repeat(K, 1)
# # # new_model = gpt.to(device)
# # # new_model.load_state_dict(torch.load("family-guy-lm-BPE-C380-E80-N9000-H7.pth"))
# # new_model.eval()

gpt_eval.eval()
with torch.no_grad():
    print("Generating text...")
    print(prompt, end="")
    out = gpt_eval.predict(test, num_tokens=500, temperature=1)

Generating text...
"Hello", said John -- " "We won't be men, I'm really very ready to best diseased by all the young Yorkers, You're not adapted of that name could be done. I woke without life and Buckkett and I began to know you can't telling the upper woman. I was far a straightium, Jhoh. Out of my attempts, heavi took the last suit on the crow, I'd have to go there, and I've come up impression to him, but I judged us and smiled brisks a large of barely occurred and saw the time I also bjected the head of side the surface with them got the Mlayman's last of antisn't a city... His little little ship dot of mass, and monitor but the alien did not have a agilea while by starting the hoverrobot, this is trying to plot the avalue-rob

In [ ]:
# """Peter: Yeah.
#   Peter: Where do you think you are? What do you think you're thinking?
#   Lois: I don't know. I'm kidding. He didn't even know you how to explain it
#   was bad. It's a time machine.
#   Stewie: Of course! Did you see my dunk was at the other room?
#   Peter: Let's see. He died in!
#   Stewie: I swear to God, he didn't know he's dead!
#   Peter: Great, huh? Can I see your dad stinks? He found the dad gets
#   firest, Lois, but we gotta be him for a family after lunch with the
#   filler. Here comes the White Lagmie Prince things]
# [closing theme music]

# Title: Da Me Life Over

# Theme Song
#   Lois: Oh, I'm sorry. They have to face a little further
#   reason I call a responsibility to jai"""

In [ ]:
# # # import torch
# # # from FlashMetal import FlashAttentionMPS

# q = torch.randn(batch_size, n_heads, block_size, 96, requires_grad=True, device="mps")
# k = torch.randn(batch_size, n_heads, block_size, 96, requires_grad=True, device="mps")
# v = torch.randn(batch_size, n_heads, block_size, 96, requires_grad=True, device="mps")

# out = FlashAttentionAutograd.apply(q,k,v)
# out.retain_grad()
# # loss
# l1 = torch.mean(out)
# l1.backward()


# dO_test = (out.grad)

# s = q @ k.transpose(-1,-2)
# s /= np.sqrt(96)

# mask = torch.tril(torch.ones_like(s)).to("mps")
# s_masked = torch.where(mask == 1, s, torch.tensor(float('-inf')).to("mps"))

# P = F.softmax(s_masked, -1)

# o_test = (torch.matmul(P, v))

# #dO = torch.randn_like(q, device='mps')
# o1 = (s_masked @ v)
# dP = torch.matmul(dO_test, v.transpose(-1, -2))
# dS = P * (dP - torch.sum(dP * P, dim=-1, keepdim=True))

# dQ_test = torch.matmul(dS, k)
# print(dQ_test)
# print(q.grad)



In [ ]:
# # print(gpt.embedding_table(torch.zeros(2, dtype=torch.long, device="mps")))

# # print(gpt.blocks[0].mhatt.batch_qkv_matrices.weight)

# q = torch.randn(1, 1, block_size, 96, requires_grad=True, device="mps")
# k = torch.randn(1, 1, block_size, 96, requires_grad=True, device="mps")
# v = torch.randn(1, 1, block_size, 96, requires_grad=True, device="mps")

# # # # # print(v.is_contiguous())



# # # for i in range(1):
# o = FlashAttentionAutograd.apply(q,k,v)

# # attention
# s = q @ k.transpose(-1,-2)
# s /= np.sqrt(96)
# # apply mask
# mask = torch.tril(torch.ones_like(s)).to("mps")
# s_masked = torch.where(mask == 1, s, torch.tensor(float('-inf')).to("mps"))

# max_vals = torch.max(s, dim=-1)[0]

# # exp_att = torch.exp(s_masked - max_vals)
# # sum_exp_att = torch.sum(exp_att, dim=-1)
# # print(max_vals)

# P = F.softmax(s_masked, -1)
# # output
# o2 = (torch.matmul(P, v))


# # print(sum_exp_att.shape)


(tensor([[[-1.0482,  1.3399,  0.0039,  ..., -0.2387, -0.1393, -1.8804],
          [-0.1157,  0.5917, -1.0879,  ..., -0.1437,  1.6654, -1.7925],
          [ 0.0839, -0.0993, -1.2655,  ...,  0.9549,  1.1826, -1.1281],
          ...,
          [-0.7517, -0.5227, -0.0832,  ...,  2.9015, -2.9129, -0.1119],
          [ 0.5305, -1.0390, -2.2558,  ...,  2.3955,  3.4465, -3.6645],
          [-0.2959, -0.2837, -1.1684,  ...,  1.5058,  2.0139, -1.0723]]],
        device='mps:0'),
 tensor([[[-0.2176,  2.2212,  3.0070,  ...,  0.6627, -1.8530,  0.0553],
          [-0.5412, -0.5362,  1.1293,  ...,  0.6604, -0.0362, -0.6014],
          [-1.3144,  0.9443,  2.0863,  ...,  1.1473,  1.3490,  3.3220],
          ...,
          [-0.8646,  1.9807,  0.4855,  ...,  0.2019, -1.4975, -0.8897],
          [-0.4465, -0.0870,  0.6601,  ...,  0.8354,  1.3764,  2.4056],
          [-0.3186,  0.3021,  0.0438,  ..., -1.4146,  0.4388, -0.7059]]],
        device='mps:0'))